In [1]:
!pip install python-telegram-bot aiohttp python-dotenv jupyter



[notice] A new release of pip is available: 24.2 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import aiohttp
import asyncio
from dotenv import load_dotenv
from telegram import Update
from telegram.ext import ApplicationBuilder, CommandHandler, MessageHandler, filters, ContextTypes

# Load environment variables
load_dotenv()

BOT_TOKEN = os.getenv("TELEGRAM_BOT_TOKEN")
OLLAMA_URL = os.getenv("OLLAMA_URL", "http://localhost:11434/api/generate")


In [ ]:
async def query_ollama(prompt: str) -> str:
    """Send a prompt to Ollama and return response."""
    payload = {
        "model": "llama3.2",
        "prompt": prompt,
        "stream": False
    }

    async with aiohttp.ClientSession() as session:
        async with session.post(f"{OLLAMA_URL}/api/generate", json=payload) as resp:
            if resp.status == 200:
                try:
                    data = await resp.json()
                    return data.get("response", "⚠ No response from model.")
                except Exception as e:
                    text = await resp.text()
                    return f"⚠ JSON parse error: {e}\nRaw: {text}"
            else:
                error_text = await resp.text()
                return f"⚠ Ollama error {resp.status}: {error_text}"

In [4]:
async def start(update: Update, context: ContextTypes.DEFAULT_TYPE):
    await update.message.reply_text(
        "🤖 *Welcome to Ollama Code Explainer Bot!*\n\n"
        "Send me any code snippet (Python, C, Java, etc.), and I’ll explain it line-by-line.\n"
        "Use /optimize to get a cleaner or faster version of your code.",
        parse_mode="Markdown"
    )


In [5]:
async def explain_code(update: Update, context: ContextTypes.DEFAULT_TYPE):
    code = update.message.text.strip()
    if not code:
        await update.message.reply_text("⚠️ Please send valid code.")
        return
    
    await update.message.reply_text("💭 Analyzing your code with Ollama, please wait...")
    
    prompt = f"Explain the following {len(code.splitlines())}-line code step-by-step:\n\n{code}"
    explanation = await query_ollama(prompt)
    
    await update.message.reply_text(f"🧩 *Explanation:*\n{explanation[:4000]}", parse_mode="Markdown")


In [6]:
async def optimize_code(update: Update, context: ContextTypes.DEFAULT_TYPE):
    if not context.args:
        await update.message.reply_text("Please send code after /optimize command.")
        return

    code = " ".join(context.args)
    await update.message.reply_text("⚙️ Optimizing your code...")

    prompt = f"Optimize the following code for readability and performance:\n\n{code}\n\nAlso explain the changes briefly."
    optimized = await query_ollama(prompt)

    await update.message.reply_text(f"🚀 *Optimized Version:*\n{optimized[:4000]}", parse_mode="Markdown")


In [7]:
app = ApplicationBuilder().token(BOT_TOKEN).build()

# Add handlers
app.add_handler(CommandHandler("start", start))
app.add_handler(CommandHandler("optimize", optimize_code))
app.add_handler(MessageHandler(filters.TEXT & ~filters.COMMAND, explain_code))


In [ ]:
import nest_asyncio
nest_asyncio.apply()

print("🤖 Bot is running... Press Ctrl+C to stop.")
await app.run_polling()



🤖 Bot is running... Press Ctrl+C to stop.


No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "c:\Users\surep\OneDrive\Desktop\ollama_code_explainer\venv\Lib\site-packages\telegram\ext\_application.py", line 1315, in process_update
    await coroutine
  File "c:\Users\surep\OneDrive\Desktop\ollama_code_explainer\venv\Lib\site-packages\telegram\ext\_handlers\basehandler.py", line 159, in handle_update
    return await self.callback(update, context)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\surep\AppData\Local\Temp\ipykernel_3752\3001009518.py", line 10, in explain_code
    explanation = await query_ollama(prompt)
                        ^^^^^^^^^^^^^^^^^^^^
TypeError: query_ollama() missing 1 required positional argument: 'task_type'
